# Importation des Librairies

In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim

## 1. Préparation des données

1.1 Importer les données Train et Test

In [19]:
import pandas as pd
import os

# Définir les chemins des fichiers CSV (à adapter selon l'emplacement des fichiers téléchargés)
train_path = '/content/train_clean.csv'
test_path = '/content/test_clean.csv'

# Vérifier si les fichiers existent
if not os.path.exists(train_path) or not os.path.exists(test_path):
    print
else:
    # Charger les fichiers CSV
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    # Afficher les 5 premières lignes pour vérification
    print("Train Dataset:")
    print(df_train.head())
    print("\nTest Dataset:")
    print(df_test.head())


Train Dataset:
   PassengerId  Survived  Pclass  Sex  Age  Fare  Embarked  IsAlone  Title
0            1         0       3    1    1     0         0        0      1
1            2         1       1    0    2     3         1        0      3
2            3         1       3    0    1     1         0        1      2
3            4         1       1    0    2     3         0        0      3
4            5         0       3    1    2     1         0        1      1

Test Dataset:
   PassengerId  Pclass  Sex  Age  Fare  Embarked  IsAlone  Title
0          892       3    1    2     0         2        1      1
1          893       3    0    2     0         0        0      3
2          894       2    1    3     1         2        1      1
3          895       3    1    1     1         0        1      1
4          896       3    0    1     1         0        0      3


1.2 Depuis le Dataframe train, charger les features d'apprentissage dans un array numpy X_alltrain, et les labels (données à prévoir) dans un array numpy y_alltrain

In [20]:
import pandas as pd
import numpy as np
import os

# Définir le chemin du fichier
train_path = "/content/train_clean.csv"

# Vérifier si le fichier existe
if os.path.exists(train_path):
    df_train = pd.read_csv(train_path)

    # Extraire les features et labels
    X_alltrain = df_train.drop(columns=["Survived", "PassengerId"]).values
    y_alltrain = df_train["Survived"].values

    print(f"X_alltrain shape: {X_alltrain.shape}")
    print(f"y_alltrain shape: {y_alltrain.shape}")

else:
    print




X_alltrain shape: (891, 7)
y_alltrain shape: (891,)


1.3 Séparer les features et les labels en deux parties (train et dev), en attribuant 10% des exemples aux données de dev. afficher les nombres de lignes et de colonnes pour les 4 arrays.

In [21]:
from sklearn.model_selection import train_test_split

# Séparation en 90% train / 10% dev
X_train, X_dev, y_train, y_dev = train_test_split(X_alltrain, y_alltrain, test_size=0.1, random_state=42, stratify=y_alltrain)

# Afficher les dimensions des 4 arrays
print(f"X_train shape: {X_train.shape}")
print(f"X_dev shape: {X_dev.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_dev shape: {y_dev.shape}")

X_train shape: (801, 7)
X_dev shape: (90, 7)
y_train shape: (801,)
y_dev shape: (90,)


1.4 Afficher les 10 premières lignes de features et les 10 premiers labels.

In [22]:
# Afficher les 10 premières lignes des features
print("10 premières lignes de X_train:")
print(X_train[:10])

# Afficher les 10 premiers labels
print("\n10 premiers labels de y_train:")
print(y_train[:10])


10 premières lignes de X_train:
[[3 1 0 3 0 0 1]
 [1 0 0 3 1 0 2]
 [3 1 1 2 2 1 1]
 [3 1 1 1 0 1 1]
 [3 1 1 0 0 1 1]
 [1 1 2 3 0 0 1]
 [1 1 3 3 0 1 1]
 [1 0 3 3 1 0 2]
 [3 1 1 1 0 1 1]
 [1 0 1 3 0 0 2]]

10 premiers labels de y_train:
[0 1 0 0 0 0 0 1 0 1]


## 2. Définition du réseau de neurone

2.1 Définir et instancier une classe Titanic Model avec les caractéristiques suivantes :
- deux  couches cachées de 50 neurones.
- deux classes en sortie : Survivant ou non
- Des fonctions d'activation RELU
- un dropout paramétrable pour les 2 couches cachées

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones) avec activation ReLU
        self.fc1 = nn.Linear(input_size, 50)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones) avec activation ReLU
        self.fc2 = nn.Linear(50, 50)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car CrossEntropyLoss inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Calcul du nombre de boucles par epoch
num_batches_per_epoch = len(X_train) // batch_size
print(f"Nombre de minibatchs par epoch: {num_batches_per_epoch}")

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

# Définition de la fonction de coût (Cross Entropy)
criterion = nn.CrossEntropyLoss()

# Définition de l'optimiseur Adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Nombre de minibatchs par epoch: 16
TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)


2.2 : Définir des paramètres de nombre d'epochs (50) et de learning_rate (0.01)

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones)
        self.fc1 = nn.Linear(input_size, 50)
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones)
        self.fc2 = nn.Linear(50, 50)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car on utilisera CrossEntropyLoss qui inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)


2.3 définir la taille du minibatch à 50. En déduire le nombre de boucle pour chaque epoch.

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones)
        self.fc1 = nn.Linear(input_size, 50)
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones)
        self.fc2 = nn.Linear(50, 50)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car on utilisera CrossEntropyLoss qui inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Calcul du nombre de boucles par epoch
num_batches_per_epoch = len(X_train) // batch_size
print(f"Nombre de minibatchs par epoch: {num_batches_per_epoch}")

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

Nombre de minibatchs par epoch: 16
TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)


2.4 Définir un fonction de coût de type CrossEntropy

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones)
        self.fc1 = nn.Linear(input_size, 50)
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones)
        self.fc2 = nn.Linear(50, 50)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car on utilisera CrossEntropyLoss qui inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Calcul du nombre de boucles par epoch
num_batches_per_epoch = len(X_train) // batch_size
print(f"Nombre de minibatchs par epoch: {num_batches_per_epoch}")

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

# Définition de la fonction de coût (Cross Entropy)
criterion = nn.CrossEntropyLoss()

Nombre de minibatchs par epoch: 16
TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)


2.5. Définir un optimizer de type Adam, sans oublier le learning rate

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones)
        self.fc1 = nn.Linear(input_size, 50)
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones)
        self.fc2 = nn.Linear(50, 50)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car on utilisera CrossEntropyLoss qui inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Calcul du nombre de boucles par epoch
num_batches_per_epoch = len(X_train) // batch_size
print(f"Nombre de minibatchs par epoch: {num_batches_per_epoch}")

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

# Définition de la fonction de coût (Cross Entropy)
criterion = nn.CrossEntropyLoss()

# Définition de l'optimiseur Adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Nombre de minibatchs par epoch: 16
TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)


## 3. Apprentissage

3.1 exécuter l'apprentissage du modèle.
- Créer une boucle sur les epochs, qui contient elel-même une boucle sur les minibatchs.
- A chaque nouvelle itération sur les epochs, mélanger les données avec la méthode shuffle.
- Tous les 5 epochs afficher la valeur de la fonction de cout

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones) avec activation ReLU
        self.fc1 = nn.Linear(input_size, 50)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones) avec activation ReLU
        self.fc2 = nn.Linear(50, 50)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car CrossEntropyLoss inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Calcul du nombre de boucles par epoch
num_batches_per_epoch = len(X_train) // batch_size
print(f"Nombre de minibatchs par epoch: {num_batches_per_epoch}")

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

# Définition de la fonction de coût (Cross Entropy)
criterion = nn.CrossEntropyLoss()

# Définition de l'optimiseur Adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Conversion des données en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# Création du DataLoader pour gérer les batchs
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Entraînement du modèle
for epoch in range(num_epochs):
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Afficher la perte tous les 5 epochs
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Nombre de minibatchs par epoch: 16
TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)
Epoch [5/50], Loss: 0.0106
Epoch [10/50], Loss: 0.2732
Epoch [15/50], Loss: 0.8971
Epoch [20/50], Loss: 0.3889
Epoch [25/50], Loss: 0.6020
Epoch [30/50], Loss: 0.1255
Epoch [35/50], Loss: 0.2926
Epoch [40/50], Loss: 0.1863
Epoch [45/50], Loss: 0.1474
Epoch [50/50], Loss: 0.7303


3.2 Calculer la précision de la prévision sur les données dev

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones) avec activation ReLU
        self.fc1 = nn.Linear(input_size, 50)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones) avec activation ReLU
        self.fc2 = nn.Linear(50, 50)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car CrossEntropyLoss inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Calcul du nombre de boucles par epoch
num_batches_per_epoch = len(X_train) // batch_size
print(f"Nombre de minibatchs par epoch: {num_batches_per_epoch}")

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

# Définition de la fonction de coût (Cross Entropy)
criterion = nn.CrossEntropyLoss()

# Définition de l'optimiseur Adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Conversion des données en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# Création du DataLoader pour gérer les batchs
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Entraînement du modèle
for epoch in range(num_epochs):
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Afficher la perte tous les 5 epochs
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Nombre de minibatchs par epoch: 16
TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)
Epoch [5/50], Loss: 0.9063
Epoch [10/50], Loss: 0.0108
Epoch [15/50], Loss: 0.8504
Epoch [20/50], Loss: 0.0632
Epoch [25/50], Loss: 0.1963
Epoch [30/50], Loss: 0.0943
Epoch [35/50], Loss: 0.9530
Epoch [40/50], Loss: 0.9914
Epoch [45/50], Loss: 0.8517
Epoch [50/50], Loss: 0.3507


3.3 Calculer prévisions sur les données de tests

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Charger les données
train_path = '/content/train_clean.csv'
test_path = '/content/test_clean.csv'

if os.path.exists(train_path) and os.path.exists(test_path):
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    # Séparation des features et labels
    X_alltrain = df_train.drop(columns=["Survived", "PassengerId"]).values
    y_alltrain = df_train["Survived"].values

    # Séparation en train/dev
    X_train, X_dev, y_train, y_dev = train_test_split(X_alltrain, y_alltrain, test_size=0.1, random_state=42, stratify=y_alltrain)

    # Extraction des features pour le test
    X_test = df_test.drop(columns=["PassengerId"]).values
else:
    raise FileNotFoundError("Les fichiers de données ne sont pas trouvés. Assurez-vous de les télécharger dans Colab.")

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones) avec activation ReLU
        self.fc1 = nn.Linear(input_size, 50)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones) avec activation ReLU
        self.fc2 = nn.Linear(50, 50)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car CrossEntropyLoss inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

# Définition de la fonction de coût (Cross Entropy)
criterion = nn.CrossEntropyLoss()

# Définition de l'optimiseur Adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Conversion des données en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_dev_tensor = torch.tensor(X_dev, dtype=torch.float32)
y_dev_tensor = torch.tensor(y_dev, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Création du DataLoader pour gérer les batchs
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Entraînement du modèle
for epoch in range(num_epochs):
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Afficher la perte tous les 5 epochs
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Évaluation du modèle sur les données de validation
def evaluate_model(model, X_dev, y_dev):
    model.eval()
    with torch.no_grad():
        outputs = model(X_dev)
        predictions = torch.argmax(outputs, dim=1)
        accuracy = (predictions == y_dev).float().mean().item()
    return accuracy

dev_accuracy = evaluate_model(model, X_dev_tensor, y_dev_tensor)
print(f"Précision sur les données de validation: {dev_accuracy * 100:.2f}%")

# Prédictions sur les données de test
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_predictions = torch.argmax(test_outputs, dim=1)

print("Prédictions sur les données de test générées.")


TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)
Epoch [5/50], Loss: 0.9951
Epoch [10/50], Loss: 0.0054
Epoch [15/50], Loss: 0.0685
Epoch [20/50], Loss: 0.2030
Epoch [25/50], Loss: 0.8004
Epoch [30/50], Loss: 0.5760
Epoch [35/50], Loss: 0.2443
Epoch [40/50], Loss: 0.1719
Epoch [45/50], Loss: 1.7841
Epoch [50/50], Loss: 0.2171
Précision sur les données de validation: 76.67%
Prédictions sur les données de test générées.


3.4 Générer le fichier résultat et l'envoyer sur kaggle
Quel est votre score et votre classement ?

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

# Charger les données
train_path = '/content/train_clean.csv'
test_path = '/content/test_clean.csv'

if os.path.exists(train_path) and os.path.exists(test_path):
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    # Séparation des features et labels
    X_alltrain = df_train.drop(columns=["Survived", "PassengerId"]).values
    y_alltrain = df_train["Survived"].values

    # Séparation en train/dev
    X_train, X_dev, y_train, y_dev = train_test_split(X_alltrain, y_alltrain, test_size=0.1, random_state=42, stratify=y_alltrain)

    # Extraction des features pour le test
    X_test = df_test.drop(columns=["PassengerId"]).values
else:
    raise FileNotFoundError("Les fichiers de données ne sont pas trouvés. Assurez-vous de les télécharger dans Colab.")

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        # Première couche cachée (50 neurones) avec activation ReLU
        self.fc1 = nn.Linear(input_size, 50)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        # Deuxième couche cachée (50 neurones) avec activation ReLU
        self.fc2 = nn.Linear(50, 50)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        # Couche de sortie (2 classes : Survivant ou Non)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car CrossEntropyLoss inclut softmax
        return x

# Définition des hyperparamètres
input_size = X_train.shape[1]  # Nombre de features
learning_rate = 0.01  # Taux d'apprentissage
num_epochs = 50  # Nombre d'époques
batch_size = 50  # Taille du minibatch

# Initialisation du modèle
model = TitanicModel(input_size)
print(model)

# Définition de la fonction de coût (Cross Entropy)
criterion = nn.CrossEntropyLoss()

# Définition de l'optimiseur Adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Conversion des données en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_dev_tensor = torch.tensor(X_dev, dtype=torch.float32)
y_dev_tensor = torch.tensor(y_dev, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Création du DataLoader pour gérer les batchs
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Entraînement du modèle
for epoch in range(num_epochs):
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Afficher la perte tous les 5 epochs
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Évaluation du modèle sur les données de validation
def evaluate_model(model, X_dev, y_dev):
    model.eval()
    with torch.no_grad():
        outputs = model(X_dev)
        predictions = torch.argmax(outputs, dim=1)
        accuracy = (predictions == y_dev).float().mean().item()
    return accuracy

dev_accuracy = evaluate_model(model, X_dev_tensor, y_dev_tensor)
print(f"Précision sur les données de validation: {dev_accuracy * 100:.2f}%")

# Prédictions sur les données de test
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_predictions = torch.argmax(test_outputs, dim=1)

print("Prédictions sur les données de test générées.")


TitanicModel(
  (fc1): Linear(in_features=7, out_features=50, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)
Epoch [5/50], Loss: 1.0703
Epoch [10/50], Loss: 0.3937
Epoch [15/50], Loss: 0.5924
Epoch [20/50], Loss: 0.0421
Epoch [25/50], Loss: 0.6659
Epoch [30/50], Loss: 0.1701
Epoch [35/50], Loss: 0.3454
Epoch [40/50], Loss: 1.1207
Epoch [45/50], Loss: 0.0309
Epoch [50/50], Loss: 0.4542
Précision sur les données de validation: 81.11%
Prédictions sur les données de test générées.


3.5 (Optionnel) Exécuter une Cross Validationd ans une boucle pour trouver les meilleures valeurs de learning rate, de keep_prob et de nombre d'epochs.

Cross_Validation avec skkearn : https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, StratifiedKFold

# Charger les données
train_path = '/content/train_clean.csv'
test_path = '/content/test_clean.csv'

if os.path.exists(train_path) and os.path.exists(test_path):
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)

    # Séparation des features et labels
    X_alltrain = df_train.drop(columns=["Survived", "PassengerId"]).values
    y_alltrain = df_train["Survived"].values

    # Extraction des features pour le test
    X_test = df_test.drop(columns=["PassengerId"]).values
else:
    raise FileNotFoundError("Les fichiers de données ne sont pas trouvés. Assurez-vous de les télécharger dans Colab.")

class TitanicModel(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(TitanicModel, self).__init__()

        self.fc1 = nn.Linear(input_size, 50)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(50, 50)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)  # Pas d'activation ici, car CrossEntropyLoss inclut softmax
        return x

# Définition des hyperparamètres
learning_rates = [0.01, 0.001]
num_epochs = 50
batch_size = 50
dropout_rates = [0.3, 0.5]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

best_accuracy = 0
best_params = {}

for lr in learning_rates:
    for dropout_rate in dropout_rates:
        fold_accuracies = []

        for train_idx, val_idx in kfold.split(X_alltrain, y_alltrain):
            X_train, X_dev = X_alltrain[train_idx], X_alltrain[val_idx]
            y_train, y_dev = y_alltrain[train_idx], y_alltrain[val_idx]

            model = TitanicModel(input_size=X_train.shape[1], dropout_rate=dropout_rate)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
            y_train_tensor = torch.tensor(y_train, dtype=torch.long)
            X_dev_tensor = torch.tensor(X_dev, dtype=torch.float32)
            y_dev_tensor = torch.tensor(y_dev, dtype=torch.long)

            dataset = TensorDataset(X_train_tensor, y_train_tensor)
            train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

            for epoch in range(num_epochs):
                for X_batch, y_batch in train_loader:
                    optimizer.zero_grad()
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    loss.backward()
                    optimizer.step()

            model.eval()
            with torch.no_grad():
                outputs = model(X_dev_tensor)
                predictions = torch.argmax(outputs, dim=1)
                accuracy = (predictions == y_dev_tensor).float().mean().item()
                fold_accuracies.append(accuracy)

        avg_accuracy = np.mean(fold_accuracies)
        print(f"LR: {lr}, Dropout: {dropout_rate}, Accuracy: {avg_accuracy:.4f}")

        if avg_accuracy > best_accuracy:
            best_accuracy = avg_accuracy
            best_params = {'learning_rate': lr, 'dropout_rate': dropout_rate}

print(f"Meilleurs paramètres : {best_params} avec une précision de {best_accuracy:.4f}")

LR: 0.01, Dropout: 0.3, Accuracy: 0.8137
LR: 0.01, Dropout: 0.5, Accuracy: 0.8171
LR: 0.001, Dropout: 0.3, Accuracy: 0.8070
LR: 0.001, Dropout: 0.5, Accuracy: 0.8036
Meilleurs paramètres : {'learning_rate': 0.01, 'dropout_rate': 0.5} avec une précision de 0.8171
